# Загрузка данных PAYKEEPER to ClickHouse

# ТЕСТИРУЕМ. АККУРАТНО С ДАТАМИ СТАРТ-ЭНД

# дозапись

In [ ]:
import json
import requests
import pandas as pd
#from ga_connector import ga_connect
import datetime
from sqlalchemy import create_engine
from pandas import DataFrame as df
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from clickhouse_key import ch_key
from clickhouse_driver import Client

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
client = Client(ch_key['db_host'],
               user=ch_key['db_user'],
               password=ch_key['db_password'],
               verify=False,
               database=ch_key['db_name'], settings={'use_numpy': True})

In [ ]:
def df(result, columns):
    col = []
    for i in columns:
        col.append(i[0])
        df = pd.DataFrame(result,columns=col)
    return df    
        
def date(date):
    date  = (datetime.datetime.strptime(date['date'][0],"%Y-%m-%d").date()+datetime.timedelta(days=1)) 
    return date

In [ ]:
result, columns = client.execute('''SELECT CAST(MAX(DATE(dt)) as String) as date 
FROM external.PAYKEEPER_CUT ''', with_column_types = True)
data = df(result, columns)
start = date(data)
start

In [ ]:
end = datetime.datetime.today().date() - datetime.timedelta(days=1)
end

In [ ]:
url = 'https://m2-vas.server.paykeeper.ru/'
q2 = f'info/payments/bydate/?start={start}&end={end}&payment_system_id[]=52&status[]=success&limit=10000'
q2 = url+q2

token = '' #удалила

headers = {
        'Authorization':  f'Basic {token}',
        "Content-Type": "application/json"
        }

x = requests.get(q2, headers= headers)
pokupka_data = pd.DataFrame(x.json())
pokupka_data.head(3)

In [ ]:
pokupka_data = pokupka_data[['id','clientid','orderid','pay_amount', 'payment_system_id','status','pending_datetime','obtain_datetime', 'success_datetime']].copy()

In [ ]:
pokupka_data['payment_system_id'] = 'ВТБ'
pokupka_data['status'] = 'Совершён'
pokupka_data['pay_amount'] = pokupka_data['pay_amount'].astype(float)
p_keeper = pokupka_data.copy()
p_keeper['dt'] = p_keeper['obtain_datetime'].apply(lambda x: datetime.datetime.strptime(x[:10], "%Y-%m-%d").date())
p_keeper['isoweek'] = p_keeper['dt'].apply(lambda x: x.isocalendar()[1])

In [ ]:
import pandas as pd
p_keeper['id'] = p_keeper['id'].astype(int)
p_keeper['pay_amount'] = p_keeper['pay_amount'].astype(float)
p_keeper['isoweek'] = p_keeper['isoweek'].astype(int)

p_keeper['clientid'] = p_keeper['clientid'].astype(str)
p_keeper['orderid'] = p_keeper['orderid'].astype(str)
p_keeper['payment_system_id'] = p_keeper['payment_system_id'].astype(str)
p_keeper['status'] = p_keeper['status'].astype(str)
p_keeper['dt'] = pd.to_datetime(p_keeper['dt'], format='%Y-%m-%d')

In [ ]:
import pandas as pd
p_keeper['pending_datetime'] = pd.to_datetime(p_keeper['pending_datetime'], format='%Y-%m-%d')
p_keeper['obtain_datetime'] = pd.to_datetime(p_keeper['obtain_datetime'], format='%Y-%m-%d')
p_keeper['success_datetime'] = pd.to_datetime(p_keeper['success_datetime'], format='%Y-%m-%d')

In [ ]:
p_keeper.info()

In [ ]:
# client.execute('''CREATE TABLE external.PAYKEEPER_CUT(id Int32, clientid String, orderid String, pay_amount Float64, 
# payment_system_id  String, status String, pending_datetime DateTime64, obtain_datetime DateTime64, success_datetime DateTime64, dt DateTime64, isoweek Int32) ENGINE = MergeTree() ORDER BY (dt, id)'''
# )

In [ ]:
client.insert_dataframe("INSERT INTO external.PAYKEEPER_CUT VALUES", p_keeper) #VALUES - DO NOT FORGET TO SET

# Загрузка данных ODS_AUTH

# дозапись

In [ ]:
f = open("ps_key.txt", "r")
postrges_key = f.read()
engine = create_engine(postrges_key)

In [ ]:
result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
FROM pg.ODS_AUTH_EVENT_CUT''', with_column_types = True)
data = df(result, columns)
start = date(data)
start

In [ ]:
q = f'''
SELECT
user_code as user_id,
CAST(role_event_code as TEXT) as role,
event_ts as date
FROM  "ODS_AUTH_USER"."EVENT" WHERE event_ts >= '{start}'
'''
ww = pd.read_sql_query(q, engine)
ww.head()

In [ ]:
group = ww.groupby(['role'], dropna=False).agg(user_id=('user_id', 'nunique')).reset_index()
group

In [ ]:
client.insert_dataframe("INSERT INTO pg.ODS_AUTH_EVENT_CUT VALUES", ww)

# Cкрипт воронки на основе views в ClickHouse

In [ ]:
re, columns = client.execute('''

WITH PAID AS
(select toISOWeek(date) as week,
       role,
       SUM(paid_users_num)  as paid_users_num,
       sum(All_paid) as All_paid,
       sum(repeat_paid) as repeat_paid,
       sum(all_paid_c) as all_paid_c,
       sum(repeat_paid_c) as repeat_paid_c
from messnerav.PUBLIC_FUNNEL_PAID_FULL
WHERE week >=1
AND toISOYear(date) = 2023
group by week, role
order by week, role),

EVENTS AS
(select
    toISOWeek(date) as week,
    role,
    sum(page_views) as page_views,
    sum(ads) as ads,
    sum(pubs) as pubs,
    count(distinct all_users) as all_users,
    count(distinct page_users) as page_users ,
    count(distinct add_users) as add_users ,
    count(distinct pbs_users) as pbs_users

from messnerav.PUBLICS_FUNNEL_EVENTS_FULL
WHERE week >=1
AND toISOYear(date) = 2023
group by week, role
order by week, role)

SELECT week, role, page_views, ads, pubs, All_paid, all_paid_c, repeat_paid, repeat_paid_c, 
all_users, page_users, add_users, pbs_users, paid_users_num


FROM EVENTS join PAID USING(week, role)''', with_column_types = True)

# datas = df(result, columns)
# datas.head(6)

In [ ]:
df_state = pd.DataFrame(re, columns=["isoweek", "role", "page_views", "ads", "pubs", "All_paid", "all_paid_c", "repeat_paid", "repeat_paid_c", "all_users", "page_users", "add_users", "pbs_users", "paid_users_num"])
df_state.head(15)

In [ ]:
df_state.isnull().sum()

In [ ]:
df = df_state.astype(np.int32).copy()

In [ ]:
df.head(3)

In [ ]:
pols= {
3 :'Физик',13 : 'ЧМАК' ,38 : 'Агентство'}
rows = [[
       'Неделя',
       'ISO неделя',
       'Роль',
       'Просмотры стр. Мои Объявления',
       'Добавления объявлений',
       'Публикация объявлений',
       'Сумма оплат',
       'Успешные оплаты',
       'Сумма повторных оплат',
       'Повторные оплаты',     
       'УП Авторизованных  Всего',
       'УП на "Мои объявления"',
       'УП Добавления объявлений',
       'УП Публикация объявлений',
       'Заплатившие пользователи'
       ]]

c = 0

for i in df.itertuples():    
    if i.isoweek == 0:
        continue
    else:
        if c % 3 == 0:
            rows.append([""]*len(rows[0]))
        ls=list(i)[1:]
        week_h = datetime.date.fromisocalendar(2023,int(i.isoweek),1)
        del ls[1]
        ls.insert(0, str(week_h) +" - "+ str(week_h+  datetime.timedelta(days = 6))) 
        ls.insert(2,str(pols[i.role]))               
        rows.append(ls)
        c+= 1

In [ ]:
import gspread
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
             'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('m2-main-cd9ed0b4e222.json', SCOPES)
gc = gspread.authorize(credentials)

In [ ]:
sh = gc.open("3. Воронка публикации объявлений")
wk = sh.worksheet('source_2023')
wk.update('A1', rows)
wk.format("1:1", {'textFormat': {'bold': True}})

# Publics for ANCH for Mondays (дозапись)

In [ ]:
def df(result, columns):
    col = []
    for i in columns:
        col.append(i[0])
        df = pd.DataFrame(result,columns=col)
    return df 

def date(date):
    date  = (datetime.datetime.strptime(date['date'][0],"%Y-%m-%d").date()+datetime.timedelta(days=1)) 
    return date

In [ ]:
result, columns = client.execute('''SELECT MAX(week) as week 
FROM messnerav.ANCH_PUBS_PAID_CR ''', with_column_types = True)
data = df(result, columns)
data

In [ ]:
# week = data.squeeze() + 1
week = 2

In [ ]:
re, columns = client.execute(f'''

WITH PAID AS
         (SELECT CAST(toISOYear(date), 'Int') AS year,
                 toISOWeek(date)              AS week,
                 sum(paid_users_num)          AS paid_users_num
          FROM messnerav.PUBLIC_FUNNEL_PAID_FULL
          WHERE ((date >= '2023-01-02') AND (toISOWeek(date) = {week}))
            AND (role IN (13, 7, 37, 38))
          GROUP BY year,
                   week
          ORDER BY year ASC,
                   week ASC),
     EVENTS AS
         (SELECT CAST(toISOYear(date), 'Int') AS year,
                 toISOWeek(date)              AS week,
                 countDistinct(pbs_users)     AS pbs_users
          FROM messnerav.PUBLICS_FUNNEL_EVENTS_FULL
          WHERE ((date >= '2023-01-02') AND (toISOWeek(date) = {week}))
            AND (role IN (13, 7, 37, 38))
          GROUP BY year,
                   week
          ORDER BY year ASC,
                   week ASC)
SELECT year,
       week,
       coalesce(pbs_users, 0)               AS pbs_users,
       coalesce(paid_users_num, 0)          AS paid_users_num,
       round(paid_users_num / pbs_users, 3) AS CR
FROM EVENTS
         LEFT JOIN PAID USING (year, week);

''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)
traf.head(5)

In [ ]:
traf['year'] = traf['year'].astype(int)
traf['week'] = traf['week'].astype(int)
traf['pbs_users'] = traf['pbs_users'].astype(int)
traf['paid_users_num'] = traf['paid_users_num'].astype(int)

In [ ]:
# client.execute(
# "CREATE TABLE messnerav.ANCH_PUBS_PAID_CR(year Int, week Int, pbs_users Nullable(Int), paid_users_num Nullable(Int), CR Nullable(Float64)) ENGINE = MergeTree() ORDER BY (year, week)"
# )

In [ ]:
client.insert_dataframe("INSERT INTO messnerav.ANCH_PUBS_PAID_CR VALUES", traf)